In [1]:
import pandas as pd
import numpy as np
import os
import sqlite3
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key) 

sk-proj-LHPdIFYw1wcQeVTI635JT3BlbkFJJRmD0xaCpJccu1A9F7A7


In [3]:
data = pd.read_csv('scenario.csv', encoding='utf-8')
data.head()

,인덱스,유형,QA
0,1,생물 보호,생물 보호 시나리오 생성해줘\n시간: 10:00\n상황: 어부들이 불법으로 산호를 ...
1,2,생물 보호,생물 보호 시나리오 생성해줘\n시간: 15:00\n상황: 관광객들이 불법으로 바다 ...
2,3,생물 보호,생물 보호 시나리오 생성해줘\n시간: 11:00\n상황: 다이버들이 불법으로 해삼을...
3,4,생물 보호,생물 보호 시나리오 생성해줘\n시간: 16:00\n상황: 어린이가 해수욕장에서 해양...
4,5,생물 보호,생물 보호 시나리오 생성해줘\n시간: 13:00\n상황: 어부들이 보호 구역에서 불...


In [4]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
database = Chroma(persist_directory="./database", embedding_function = embeddings )

In [5]:
# 각 행의 데이터를 Document 객체로 변환 
documents = [Document(page_content=text) for text in data['QA'].tolist()]

# 데이터프레임에서 문서 추가
database.add_documents(documents)

['6ac21b3b-4b01-49cb-a0bd-cbed04c3ff6f',
 'f11a62d2-85d4-410e-998e-e596d94bc439',
 '4ae58a32-bb7b-48f5-8d98-a0ec785719f1',
 'd799fda0-ef0b-4131-9eb3-0430c82af94a',
 '8c51e1d6-cd8f-45fa-af26-aa7665bd88e1',
 '24294b62-ef7b-4161-8d4a-cd29e634a682',
 'ee0eadb9-4b59-4b9e-961d-87ceb7180b67',
 'f1e54ea3-11b1-47f8-85cb-5199ca3b19a7',
 '228c1988-63c8-436d-9828-b9d6f54c8790',
 'f9f410f7-c725-41e1-92a7-4684d85c8964',
 '39a3cce2-1695-43ab-ac77-7bf9764061ee',
 'e6017f3a-b712-4500-a626-24d1d75183c3',
 '4c71fd51-2d89-45bc-8dde-496aa54e2df1',
 '92942853-8b29-4cec-aae8-fbf82246c43e',
 '013bf172-34b7-4ceb-9a71-7536e61e21cb',
 'd9d1b09d-9db2-4b1b-a7cb-7359b65dff2e',
 '1774523f-d43b-4bdc-8f79-2a4d10d0c183',
 '81ceb71a-d9ec-438c-bb61-be7e00ce4de3',
 '2855ea89-655f-4183-a04c-0290bd5dc182',
 'bf94e8f6-5056-4917-9bf9-04f3640abe88',
 'c8dadffa-735b-4a04-9827-e618ea4b71ae',
 '1c7d6db2-8120-4c82-8a66-c765ac95d014',
 '1d0ce3d6-90c9-41fd-a1be-b97254a15510',
 '0763f0da-8a8c-499d-8b4b-da920d8daba9',
 '06d533be-1a95-

In [6]:
# 입력된 데이터 조회
database.get()

{'ids': ['013bf172-34b7-4ceb-9a71-7536e61e21cb',
  '01f4f303-c381-4570-a3a1-2425da6a58d4',
  '03053885-062e-4480-8a7c-89528c1c29d4',
  '03666cb9-f62b-4b72-a085-dd6e61cd4ae6',
  '03cacb8a-4822-4355-83d9-c87411c3070f',
  '04b034df-5ed0-4085-8c40-0ac3cc297c7f',
  '06d533be-1a95-4239-b0a8-6d659c61dbcb',
  '0763f0da-8a8c-499d-8b4b-da920d8daba9',
  '0c6b9d20-34ca-40ad-9517-62a80d2454e0',
  '0eb9041e-ff03-480e-96ef-7146b0cbc205',
  '100bd034-243d-40fa-8942-5776353682c7',
  '128b4d25-24ee-4d22-b9e2-0107cbbf2de8',
  '13aa2778-a6fc-4c29-aa12-1c48da9e4eae',
  '14dbd3d4-b4c8-42e8-9b5c-fb3e3b0de138',
  '151e1217-b7ea-44f8-91ec-c8230fb4dcfd',
  '1774523f-d43b-4bdc-8f79-2a4d10d0c183',
  '1a4d9741-dd34-47b3-9c81-86802458ae9e',
  '1a5174cb-81a2-49b5-b8f4-321d11000c5e',
  '1a98fa87-10ec-4170-aa98-4d1068fe1440',
  '1b34a714-bd7e-48ed-b8be-450841d73640',
  '1c7d6db2-8120-4c82-8a66-c765ac95d014',
  '1ceb9f8f-a8e6-40f4-bf35-64d512c3f873',
  '1d0ce3d6-90c9-41fd-a1be-b97254a15510',
  '201c5fdd-875b-4deb-bdb7-

In [7]:
chat = ChatOpenAI(model="gpt-3.5-turbo")

k=20
retriever = database.as_retriever(search_kwargs={"k": k})

# 대화 메모리 생성
memory = ConversationBufferMemory(memory_key="chat_history", input_key="question", output_key="answer", 
                                  return_messages=True)

# ConversationalRetrievalQA 체인 생성
qa = ConversationalRetrievalChain.from_llm(llm=chat, retriever=retriever, memory=memory, 
                                           return_source_documents=True,  output_key="answer")

In [8]:
query = '유형: 수상레저, 상황: 제트스키 탑승 중 한 명이 제트스키에서 떨어져 다리 부상을 당함.'
result = qa(query)

print(result["answer"])

해당 상황에서의 수상레저 시나리오는 다음과 같습니다:

- **시간**: 10:00
- **상황**: 제트스키 탑승 중 한 명이 제트스키에서 떨어져 다리 부상을 당함.
- **절차**:
  - **신고 접수**:
    - **신고자**: 제트스키 운영 직원
    - **신고 대상**: 해양 경찰 구조 센터
    - **신고 내용**: 다리 부상자 발생, 위치, 부상자의 상태
    - **시간 소요**: 1분
  - **초기 대응**:
    - **출동 인원**: 해양 경찰 구조팀 4명 (구조 보트 2명, 응급 처치 팀 2명)
    - **준비 작업**: 구조 보트 준비, 응급 처치 장비 준비
    - **시간 소요**: 2분
  - **현장 도착**:
    - **구조 보트 도착 시간**: 5분 이내
    - **출동 경로**: 해안에서 부상자 위치까지 직선 경로
  - **구조 작업**:
    - **구조 대원**: 2명 (1명 보트 운전, 1명 부상자 접근 및 구조)
    - **구명조끼 착용 및 구조**: 3분
    - **부상자를 보트로 옮김**: 2분
  - **응급 처치**:
    - **처치 인원**: 응급 처치 팀 2명
    - **처치 내용**: 다리 부상 응급 처치, 출혈 관리
    - **시간 소요**: 4분
  - **후속 조치**:
    - **이송 경로**: 해변 응급처치소
    - **이송 시간**: 3분
    - **추가 의료 지원**: 필요 시 추가 처치 (의사 또는 추가 의료팀 지원)
  - **목표**: 총 20분 이내에 부상자 구조 및 응급 처치 완료

이것이 수상레저 시나리오의 전체 과정입니다.
